# 5.3.2 NGSolve - PETSc interface
We use the ngs2petsc interface to map vectors and matrices between NGSolve and PETSc

In [ ]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)

In [ ]:
%%px
from ngsolve import *
from netgen.occ import unit_square
comm = MPI.COMM_WORLD

ngmesh = unit_square.GenerateMesh(maxh=0.1, comm=comm)
    
for l in range(2):
    ngmesh.Refine()
mesh = Mesh(ngmesh)

The Python-module ngsolve.ngs2petsc provides functionality to transfer vectors and matrices between NGSolve and Python. 

Make sure that the ipyparallel server can import the module, e.g. by starting the cluster in the current directory.

In [ ]:
%%px
import ngsolve.ngs2petsc as n2p
import petsc4py.PETSc as psc

In [ ]:
%%px
fes = H1(mesh, order=1, dirichlet="left|bottom")
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx+u*v*ds).Assemble()
f = LinearForm(x*v*dx).Assemble()
gfu = GridFunction(fes)

The function CreatePETScMatrix takes an NGSolve matrix, and creates a PETSc matrix from it. A VectorMapping object can map vectors between NGSolve and PETSc. 

In [ ]:
%%px
psc_mat = n2p.CreatePETScMatrix(a.mat, fes.FreeDofs())
vecmap = n2p.VectorMapping (a.mat.row_pardofs, fes.FreeDofs())

Create PETSc-vectors fitting to the matrix

In [ ]:
%%px
psc_f, psc_u = psc_mat.createVecs()

setting up the parallel Krylov-space solver ....

In [ ]:
%%px
ksp = psc.KSP()
ksp.create()
ksp.setOperators(psc_mat)
ksp.setType(psc.KSP.Type.CG)
ksp.setNormType(psc.KSP.NormType.NORM_NATURAL)
ksp.getPC().setType("gamg")
ksp.setTolerances(rtol=1e-6, atol=0, divtol=1e16, max_it=400)

moving vectors between NGSolve and PETSc, and solve:

In [ ]:
%%px
vecmap.N2P(f.vec, psc_f)
ksp.solve(psc_f, psc_u)   
vecmap.P2N(psc_u, gfu.vec);

In [ ]:
gfu = c[:]["gfu"]
from ngsolve.webgui import Draw

In [ ]:
Draw (gfu[0])

### PETSc preconditioner for NGSolve
Next we create a PETSc preconditioner, and wrap it into an NGSolve preconditioner:

In [ ]:
%%px
a = BilinearForm(grad(u)*grad(v)*dx+u*v*ds)
# pre = Preconditioner(a, "petsc", pctype="gamg", levels=10)
pre = Preconditioner(a, "gamg")
a.Assemble();

and use it in an NGSolve - CGSolver:

In [ ]:
%%px
from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates=comm.rank==0)
gfu.vec.data = inv * f.vec

In [ ]:
gfu = c[:]["gfu"]
Draw (gfu[0]);